# Data Preparation

In [1]:
import logging
import os
import pandas as pd

from bs4 import BeautifulSoup

In [2]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[logging.FileHandler("data_preparation.log"), logging.StreamHandler()],
)

In [3]:
PATH_TO_HTML_REVIEW_SAMPLE = (
    "../data-mining/html/cuiaba_companies/Tecnomapas_page_1.html"
)

In [4]:
PATH_TO_HTML_REVIEWS = "./html"

In [5]:
GLASSDOOR_REVIEWS_CSV_PATH = "./glassdoor_reviews.csv"

In [6]:
DATA_FRAME_COLUMNS = [
    "review_id",
    "company",
    "employee_role",
    "employee_detail",
    "rating_text",
    "review_date",
    "star_rating",
    "sentiment",
]

In [7]:
def get_rating_text(rating_div):
    rating_p = rating_div.find_all(
        "p", {"class": "review-details__review-details-module__isCollapsed"}
    )
    if len(rating_p) == 0:
        rating_p = rating_div.find_all(
            "p", {"class": "review-details__review-details-module__isExpanded"}
        )

    if len(rating_p) > 0:
        return rating_p[0].get_text()

    logging.warning("Texto da avaliação não encontrado!")
    return ""

In [8]:
def get_reviews_data(company, reviews):
    reviews_data = []
    for review in reviews:
        # Id
        review_id_li = review["id"]
        if review_id_li:
            review_id = review_id_li.split("_")[-1]
            logging.warning(f"Avaliação {review_id} encontrada!")
        else:
            logging.warning("Id da avaliação não encontrado!")

        # Date
        review_date_span = review.find(
            "span", class_="timestamp__timestamp-module__reviewDate"
        )
        if review_date_span:
            review_date = review_date_span.get_text()
        else:
            logging.warning("Data da avaliação não encontrada!")

        # Employee role
        employee_role_span = review.find(
            "span", class_="review-details__review-details-module__employee"
        )
        if employee_role_span:
            employee_role = employee_role_span.get_text()
        else:
            logging.warning("Cargo do avaliador não encontrado!")

        # Employee detail
        employee_detail_div = review.find(
            "div", {"class": "review-details__review-details-module__employeeDetails"}
        )
        if employee_detail_div:
            employee_detail = employee_detail_div.get_text()
        else:
            logging.warning("Detalhe do avaliador não encontrado!")

        # Star rating
        star_rating_spans = review.find_all(
            "span", {"class": "review-details__review-details-module__overallRating"}
        )
        if star_rating_spans:
            star_rating_span = star_rating_spans[0]
            if star_rating_span:
                star_rating = float(star_rating_span.get_text().replace(",", "."))
            else:
                logging.warning("Nota de avaliação não foi encontrada!")
        else:
            logging.warning("Nenhuma nota de avaliação foi encontrada!")

        # Pro text
        pro_rating_divs = review.find_all(
            "div", {"class": "review-details__review-details-module__pro"}
        )
        if pro_rating_divs:
            pro_rating_div = pro_rating_divs[0]
            if pro_rating_div:
                logging.info("Buscando texto positivo da avaliação...")
                pro_rating_text = get_rating_text(pro_rating_div)
            else:
                logging.warning("Div da avaliação positiva não encontrada!")
        else:
            logging.warning("Nenhuma div de avaliação positiva foi encontrado!")

        # Con text
        con_rating_divs = review.find_all(
            "div", {"class": "review-details__review-details-module__con"}
        )
        if con_rating_divs:
            con_rating_div = con_rating_divs[0]
            if con_rating_div:
                logging.info("Buscando texto negativo da avaliação...")
                con_rating_text = get_rating_text(con_rating_div)
            else:
                logging.warning("Div da avaliação negativa não encontrada!")
        else:
            logging.warning("Nenhuma div de avaliação negativa foi encontrado!")

        # Append review data
        reviews_data.append(
            [
                review_id,
                company,
                employee_role,
                employee_detail,
                pro_rating_text,
                review_date,
                star_rating,
                1,  # positive
            ]
        )
        reviews_data.append(
            [
                review_id,
                company,
                employee_role,
                employee_detail,
                con_rating_text,
                review_date,
                star_rating,
                0,  # negative
            ]
        )

    return reviews_data

In [9]:
if os.path.exists(GLASSDOOR_REVIEWS_CSV_PATH):
    reviews_df = pd.read_csv(GLASSDOOR_REVIEWS_CSV_PATH)
else:
    reviews_df = pd.DataFrame(columns=DATA_FRAME_COLUMNS)

In [10]:
reviews_df.head(2)

,review_id,company,employee_role,employee_detail,rating_text,review_date,star_rating,sentiment
0,82630669,Tecnomapas,Recepcionista,"Ex-funcionário(a), mais de um ano","Companheirismo entre os colegas, oportunidade ...",15 de dez. de 2023,5.0,1
1,82630669,Tecnomapas,Recepcionista,"Ex-funcionário(a), mais de um ano",Não tive nenhum ponto negativo,15 de dez. de 2023,5.0,0


In [11]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   review_id        54 non-null     int64  
 1   company          54 non-null     object 
 2   employee_role    54 non-null     object 
 3   employee_detail  54 non-null     object 
 4   rating_text      54 non-null     object 
 5   review_date      54 non-null     object 
 6   star_rating      54 non-null     float64
 7   sentiment        54 non-null     int64  
dtypes: float64(1), int64(2), object(5)
memory usage: 3.5+ KB


In [ ]:
for filename in os.listdir(PATH_TO_HTML_REVIEWS):
    if filename.endswith(".html"):
        file_path = os.path.join(PATH_TO_HTML_REVIEWS, filename)

        logging.warning(f"Abrindo o arquivo: {file_path}")
        with open(file_path, "r", encoding="utf-8") as f:
            html_page = "".join(f.readlines())
            soup = BeautifulSoup(html_page, "html.parser")

            if soup.title:
                if soup.title.get_text() != "Security | Glassdoor":
                    company_p = soup.find_all("p", {"class": "employerName"})[0]
                    company = company_p.get_text()

                    logging.warning(f"Buscando Avaliações da Empresa: {company}")

                    reviews = soup.find_all(
                        "li", id=lambda x: x and x.startswith("empReview_")
                    )

                    reviews_data = get_reviews_data(company, reviews)

                    df = pd.DataFrame(reviews_data, columns=DATA_FRAME_COLUMNS)

                    for index, row in df.iterrows():
                        new_review_id = int(row["review_id"])
                        if new_review_id not in reviews_df["review_id"].values:
                            print(row)
                            reviews_df = pd.concat(
                                [reviews_df, df],
                                ignore_index=True,
                            )

                            # Since we have two comments with same id for each review,
                            # we must concat the new review df only once
                            break

In [13]:
reviews_df.head(2)

,review_id,company,employee_role,employee_detail,rating_text,review_date,star_rating,sentiment
0,82630669,Tecnomapas,Recepcionista,"Ex-funcionário(a), mais de um ano","Companheirismo entre os colegas, oportunidade ...",15 de dez. de 2023,5.0,1
1,82630669,Tecnomapas,Recepcionista,"Ex-funcionário(a), mais de um ano",Não tive nenhum ponto negativo,15 de dez. de 2023,5.0,0


In [14]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2532 entries, 0 to 2531
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   review_id        2532 non-null   object 
 1   company          2532 non-null   object 
 2   employee_role    2532 non-null   object 
 3   employee_detail  2532 non-null   object 
 4   rating_text      2532 non-null   object 
 5   review_date      2532 non-null   object 
 6   star_rating      2532 non-null   float64
 7   sentiment        2532 non-null   int64  
dtypes: float64(1), int64(1), object(6)
memory usage: 158.4+ KB


In [15]:
reviews_df

,review_id,company,employee_role,employee_detail,rating_text,review_date,star_rating,sentiment
0,82630669,Tecnomapas,Recepcionista,"Ex-funcionário(a), mais de um ano","Companheirismo entre os colegas, oportunidade ...",15 de dez. de 2023,5.0,1
1,82630669,Tecnomapas,Recepcionista,"Ex-funcionário(a), mais de um ano",Não tive nenhum ponto negativo,15 de dez. de 2023,5.0,0
2,74420027,Tecnomapas,Analista Desenvolvedor,Ex-freelancer,Equipe bem prestativa e ótima de se trabalhar.,11 de mar. de 2023,4.0,1
3,74420027,Tecnomapas,Analista Desenvolvedor,Ex-freelancer,Modo home office ainda tem que ser melhorado.,11 de mar. de 2023,4.0,0
4,60212043,Tecnomapas,Funcionário confidencial,"Ex-funcionário(a), menos de um ano",Única vantagem era o trabalho ser home office,24 de fev. de 2022,1.0,1
...,...,...,...,...,...,...,...,...
2527,35726303,Totem Treinamento e Consultoria em TI,Funcionário confidencial,"Funcionário(a) atual, menos de um ano",Local de trabalho é muito pequeno.,8 de set. de 2020,5.0,0
2528,42894163,Totem Treinamento e Consultoria em TI,Analista De Sistemas,Funcionário(a) atual,Bom ambiente de trabalho.\n\nEquipe bastante c...,22 de fev. de 2021,5.0,1
2529,42894163,Totem Treinamento e Consultoria em TI,Analista De Sistemas,Funcionário(a) atual,Não tenho nenhum ponto negativo sobre a empresa.,22 de fev. de 2021,5.0,0
2530,45976432,Totem Treinamento e Consultoria em TI,Analista De Sistemas,"Funcionário(a) atual, mais de 3 anos","Oportunidade para iniciantes, muitos projetos.",23 de abr. de 2021,2.0,1


In [16]:
reviews_df.to_csv("glassdoor_reviews.csv", index=False)